# FIT5148 - Distributed Databases and Big Data

# Assignment 2 TaskC2 - Solution Workbook

**Your details:**
- Name: Zhiqing Shu
- Student ID: 28217551

- Name: Yicheng Zhang
- Student ID: 27699641

Let's get started!

**2.Write a streaming application in Apache Spark with local Streaming Context with two execution threads and a batch interval of five seconds. The application will:**

* a. Receive streaming data from the streams generated by the stream server.
* b. Process the streams to ensure the data can be uploaded into the MongoDB according to the data model you have designed in Task B1.
* c. Upload the result of processing in Task C2b to MongoDB.


In [1]:
# Mongodb packages
import pymongo
from pymongo import MongoClient
# Spark packages
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

from pprint import pprint

**Define the client port**

In [2]:
def client():
    # Create global variable can be used in different functions
    global datalist
    
    #Use Mongodb
    client = MongoClient ()
    db = client.TaskB_db
    
    # Configure spark data streaming
    # 5 seconds as window size
    bacth_interval = 5
    
    # 2 execution threads
    sc = SparkContext(master="local[2]",appName = "Store_Data")
    ssc = StreamingContext(sc,bacth_interval)
    # Get socket data at port 9999
    ds = ssc.socketTextStream("localhost", 9999)
    
    # split the datastream line by line
    # flatMap is a one-to-many DStream operation that creates a new DStream 
    # by generating multiple new records from each record in the source DStream.
    lines = ds.flatMap(lambda line: line.split("\r\n"))
    
    # Split the string data to a list
    def splitString(datalist): 
        for i in range(0,len(datalist)):       
            templist = datalist[i].split(',')
            result = []
            for element in templist:
                result.append(element)
            datalist[i] = result
        return datalist
    
    # Using the python data structure process the documents before storing in mongodb
    def transferToDoc(datalist): 
        climateList = []
        fireList = []
        # transfer string to desired type and store them to a dictionary
        for i in range(0,len(datalist)):
            tempdict = {}
            # Store the climate data
            if datalist[i][1] == 'cdata':
                tempdict['Date'] = datalist[i][0]
                tempdict['Station'] = int(datalist[i][2].strip())
                tempdict['Air Temperature(Celcius)'] = int(datalist[i][3].strip())
                tempdict['Relative Humidity'] = float(datalist[i][4].strip())
                tempdict['WindSpeed (knots)'] = float(datalist[i][5].strip())
                tempdict['Max Wind Speed'] = float(datalist[i][6].strip())
                tempdict['MAX'] = datalist[i][7].strip()
                tempdict['MIN'] = datalist[i][8].strip()
                tempdict['Precipitation'] = datalist[i][9].strip()
                datalist[i] = tempdict
                climateList.append(tempdict)
            # Store the fire data
            else:
                tempdict['Date'] = datalist[i][0]
                tempdict['Latitude'] = float(datalist[i][2].strip())
                tempdict['Longitude'] = float(datalist[i][3].strip())
                tempdict['Surface Temperature (kelvin)'] = float(datalist[i][4].strip())
                tempdict['Power'] = float(datalist[i][5].strip())
                tempdict['Confidence'] = int(datalist[i][6].strip())
                tempdict['Surface Temperature (Celcius)'] = int(datalist[i][7].strip())
                datalist[i] = tempdict
                fireList.append(tempdict)
                
        # If there is a climate data in the batch
        if len(climateList) != 0:
            # Store the fire data embeded in the climate data
            for i,val in enumerate(climateList):
                climateList[i]['fire'] = []
                for fire in fireList:
                    if climateList[i]['Date'] == fire['Date']:
                        templist = climateList[i]['fire']
                        templist.append(fire)
                        climateList[i]['fire'] = templist
            # Drop unnecessary field
            for item in climateList:
                for fire in item['fire']:
                    fire.pop('Date')
            # Return the climate data after processing
            return climateList
        # If there is no cliamte data in the batch
        else:
            newFire = []
            # Merge the Fire data based on Date
            for item in fireList:
                tempdict = {'Date':item['Date'],'Fire':[]}
                if tempdict not in newFire:
                    newFire.append(tempdict)
            for item in newFire:
                for record in fireList:
                    if item['Date'] == record['Date']:
                        item['Fire'].append(record)
            for item in newFire:
                for fire in item['Fire']:
                    fire.pop('Date')
            # Return data list  after processing
            return newFire
    
    # Process rdd, Resilient Distributed Datasets, collect the list, 
    # using the functions defined before
    def process(rdd):
        # Collect all the data in the batch
        # first bring the RDD to the driver node thus
        batch = rdd.collect()  
        # Transfer the string data to list data
        split_list = splitString(batch) 
        # Transfer the list to dictionaries
        datalist = transferToDoc(split_list)
        # Insert the results to mongodb
        db.taskC.insert_many(datalist)
        
    # Drop collection First, In case duplications
    db.taskC.drop() 
    
    # Do the process function for every rdd in the datastream.
    lines.foreachRDD(process)
    # Start socketTextStream.
    ssc.start()
    
    try:
        ssc.awaitTermination()
    except KeyboardInterrupt:
        ssc.stop()
        sc.stop()

    ssc.stop()
    sc.stop()

In [3]:
client()

**3. Testing: Use the python application from Task B (as2TaskB.ipynb​) to show thestreamed data have been added into the database successfully.**

In [4]:
client = MongoClient ()
db = client.TaskB_db

The date in this task is real-time timestemp, we cannot using exactly the same query in task B, however, we can test the data using modfied version query.

A2. Find climate data(just first 10 climate data).

In [5]:
cursor_B2 = db.taskC.find({},{'_id':0,'fire':0}).limit(10)

In [6]:
for doc in cursor_B2:
    pprint(doc)

{'Air Temperature(Celcius)': 19,
 'Date': '2018-05-21T02:26:24',
 'MAX': '72.0*',
 'MIN': '61.9*',
 'Max Wind Speed': 11.1,
 'Precipitation': '0.00I',
 'Relative Humidity': 56.8,
 'Station': 948700,
 'WindSpeed (knots)': 7.9}
{'Air Temperature(Celcius)': 15,
 'Date': '2018-05-21T02:26:25',
 'MAX': '',
 'MIN': '58.3',
 'Max Wind Speed': 13.0,
 'Precipitation': '0.02G',
 'Relative Humidity': 50.7,
 'Station': 948700,
 'WindSpeed (knots)': 9.2}
{'Air Temperature(Celcius)': 16,
 'Date': '2018-05-21T02:26:26',
 'MAX': '68.4*',
 'MIN': '54.3*',
 'Max Wind Speed': 15.0,
 'Precipitation': '0.00G',
 'Relative Humidity': 53.6,
 'Station': 948700,
 'WindSpeed (knots)': 8.1}
{'Air Temperature(Celcius)': 24,
 'Date': '2018-05-21T02:26:27',
 'MAX': '87.3*',
 'MIN': '54',
 'Max Wind Speed': 14.0,
 'Precipitation': '0.00I',
 'Relative Humidity': 61.6,
 'Station': 948700,
 'WindSpeed (knots)': 7.7}
{'Air Temperature(Celcius)': 24,
 'Date': '2018-05-21T02:26:28',
 'MAX': '85.5*',
 'MIN': '65.7*',
 'Max 

A3. Find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C (just first 10 records).

In [7]:
# Unwind fire to ensure retrieve fire data as stand-alone record
# Match surface temperature between 65 and 100
# Project specified field
# Find from first 10 records in this situation, for testing
cursor_B3 = db.taskC.aggregate([{"$limit" : 10},\
                         {"$unwind": "$fire"},\
                         {"$match": {"fire.Surface Temperature (Celcius)": { "$gte":65,"$lte":100}}},\
                         {"$project":{'fire.Latitude':1,'fire.Longitude': 1,'fire.Confidence':1,\
                          'fire.Surface Temperature (Celcius)':1,'_id':0}}])

In [8]:
for doc in cursor_B3:
    pprint(doc)

{'fire': {'Confidence': 78,
          'Latitude': -37.966,
          'Longitude': 145.051,
          'Surface Temperature (Celcius)': 68}}
{'fire': {'Confidence': 86,
          'Latitude': -35.543,
          'Longitude': 143.316,
          'Surface Temperature (Celcius)': 67}}
{'fire': {'Confidence': 93,
          'Latitude': -37.875,
          'Longitude': 142.51,
          'Surface Temperature (Celcius)': 73}}
{'fire': {'Confidence': 95,
          'Latitude': -37.613,
          'Longitude': 149.305,
          'Surface Temperature (Celcius)': 75}}
{'fire': {'Confidence': 90,
          'Latitude': -37.624,
          'Longitude': 149.314,
          'Surface Temperature (Celcius)': 66}}
{'fire': {'Confidence': 93,
          'Latitude': -38.057,
          'Longitude': 144.211,
          'Surface Temperature (Celcius)': 73}}
{'fire': {'Confidence': 92,
          'Latitude': -37.95,
          'Longitude': 142.366,
          'Surface Temperature (Celcius)': 70}}
{'fire': {'Confidence': 100,


A5.Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100 (just first 10 records).

In [9]:
# Unwind fire to ensure retrieve fire data as stand-alone record
# Match surface confidence between 80 and 100
# Project specified field
# Find from first 10 records in this situation, for testing
cursor_B5 = db.taskC.aggregate([{"$limit" : 10},\
                                  {"$unwind":"$fire"},\
                                  {"$match":{"fire.Confidence":{ "$gte":80,"$lte":100}}},\
                                  {"$project":{'fire.Datetime':1,'AirTemperature(Celcius)':1,\
                                   'fire.Confidence':1,'fire.Surface Temperature (Celcius)':1,'_id':0}}])

In [10]:
for doc in cursor_B5:
    pprint(doc)

{'fire': {'Confidence': 82, 'Surface Temperature (Celcius)': 63}}
{'fire': {'Confidence': 86, 'Surface Temperature (Celcius)': 67}}
{'fire': {'Confidence': 80, 'Surface Temperature (Celcius)': 54}}
{'fire': {'Confidence': 94, 'Surface Temperature (Celcius)': 43}}
{'fire': {'Confidence': 93, 'Surface Temperature (Celcius)': 73}}
{'fire': {'Confidence': 84, 'Surface Temperature (Celcius)': 55}}
{'fire': {'Confidence': 95, 'Surface Temperature (Celcius)': 75}}
{'fire': {'Confidence': 90, 'Surface Temperature (Celcius)': 66}}
{'fire': {'Confidence': 93, 'Surface Temperature (Celcius)': 73}}
{'fire': {'Confidence': 82, 'Surface Temperature (Celcius)': 55}}
{'fire': {'Confidence': 82, 'Surface Temperature (Celcius)': 64}}
{'fire': {'Confidence': 92, 'Surface Temperature (Celcius)': 42}}
{'fire': {'Confidence': 92, 'Surface Temperature (Celcius)': 70}}
{'fire': {'Confidence': 83, 'Surface Temperature (Celcius)': 60}}
{'fire': {'Confidence': 87, 'Surface Temperature (Celcius)': 61}}
{'fire': {

A6. Find top 10 records with highest surface temperature (°C).

In [11]:
# Unwind fire to ensure retrieve fire data as stand-alone record
# Sort by surface temperature in descending order(max to min)
# Project specified field
# Find from first 10 records in this situation, for testing
cursor_B6 = db.taskC.aggregate([{"$limit":10},\
                                {"$unwind": "$fire"},\
                                {"$sort": {"fire.Surface Temperature (Celcius)":-1}},\
                                {"$limit":10},\
                                {"$project":{'_id':0}}])

In [12]:
for doc in cursor_B6:
    pprint(doc)

{'Air Temperature(Celcius)': 16,
 'Date': '2018-05-21T02:26:26',
 'MAX': '68.4*',
 'MIN': '54.3*',
 'Max Wind Speed': 15.0,
 'Precipitation': '0.00G',
 'Relative Humidity': 53.6,
 'Station': 948700,
 'WindSpeed (knots)': 8.1,
 'fire': {'Confidence': 95,
          'Latitude': -37.613,
          'Longitude': 149.305,
          'Power': 160.9,
          'Surface Temperature (Celcius)': 75,
          'Surface Temperature (kelvin)': 348.2}}
{'Air Temperature(Celcius)': 24,
 'Date': '2018-05-21T02:26:27',
 'MAX': '87.3*',
 'MIN': '54',
 'Max Wind Speed': 14.0,
 'Precipitation': '0.00I',
 'Relative Humidity': 61.6,
 'Station': 948700,
 'WindSpeed (knots)': 7.7,
 'fire': {'Confidence': 93,
          'Latitude': -38.057,
          'Longitude': 144.211,
          'Power': 48.8,
          'Surface Temperature (Celcius)': 73,
          'Surface Temperature (kelvin)': 346.3}}
{'Air Temperature(Celcius)': 16,
 'Date': '2018-05-21T02:26:26',
 'MAX': '68.4*',
 'MIN': '54.3*',
 'Max Wind Speed': 15.0,


A8. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [13]:
# Unwind fire to ensure retrieve fire data as stand-alone record
# Count only date has fire
# group by date and caculate Average
# sort by date
# Project specified field
# Find from first 10 records in this situation, for testing
cursor_B8 = db.taskC.aggregate([{"$limit" : 10},\
                                  {"$unwind": "$fire"},\
                                  {"$match": {"fire" : {"$ne": []}}},\
                                  {"$group":{'_id':'$Date','Avg Temperature':{'$avg':"$fire.Surface Temperature (Celcius)"}}},\
                                  {"$sort":{'_id':1}},\
                                  {"$project":{'_id':0,"Date":'$_id','Avg Temperature':1}}])

In [14]:
for doc in cursor_B8:
    pprint(doc)

{'Avg Temperature': 61.0, 'Date': '2018-05-21T02:26:24'}
{'Avg Temperature': 41.6, 'Date': '2018-05-21T02:26:25'}
{'Avg Temperature': 64.8, 'Date': '2018-05-21T02:26:26'}
{'Avg Temperature': 58.4, 'Date': '2018-05-21T02:26:27'}
{'Avg Temperature': 47.4, 'Date': '2018-05-21T02:26:28'}
{'Avg Temperature': 50.4, 'Date': '2018-05-21T02:26:29'}
{'Avg Temperature': 57.4, 'Date': '2018-05-21T02:26:30'}
{'Avg Temperature': 50.6, 'Date': '2018-05-21T02:26:31'}
{'Avg Temperature': 47.6, 'Date': '2018-05-21T02:26:32'}
{'Avg Temperature': 36.0, 'Date': '2018-05-21T02:26:33'}
